In [4]:
import nltk
import ssl
import string

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context


nltk.download('stopwords')
nltk.download('punkt')
from nytimesarticle import articleAPI
api = articleAPI('API KEY HERE')
import requests
import re
from bs4 import BeautifulSoup
import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siddharthpandey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/siddharthpandey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
import time
import datetime
import os

now = datetime.datetime.now()

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)



#topics = ["cybersecurity","hack","cybercrime"]
topics = ["molestation","harassment"]
for topic in (topics):
    newsUrls = []
    #print(len(newsUrls))
    print("TOPIC NOW: ")
    print(topic)
    for i in range(0,10):
       # print("LOOP: ")
       # print(i)
        articles = api.search( q = "molestation", fq = {'source':['The New York Times']},  page=str(i))
        try:
            #print("RECEIVING DATA...")
            newsUrls = newsUrls + articles['response']['docs']
            time.sleep(1)
        except:
            pass
    #print("---ONE TOPIC END---")
    print("EXTRACTING DATA")
    allUrls=[]
    for j in newsUrls:
        allUrls.append(j['web_url'])
    
    print(len(allUrls))
    print("WRITING DATA TO FILE...")
    finalData = []
    co=0
    sum=0
    for i in allUrls:
        #print(i)
        response=requests.get(i)
        #print(response.status_code)’
        stripped = re.sub('<[^<]+?>', '', response.text)
        soup = BeautifulSoup(response.text, 'html.parser')
        res=""
        for i in soup.findAll('p'):
            res=res+str(i)

        co = co+1
        #print("*********the article ends here****************")
        content = remove_tags(res)
        content = content.replace("Advertisement","")

        finalString = ""
        punctuations = '''!()-[]{};:'’"\,.,”“.“‘<>—./?@#$%^&*_~'''
        for character in content:
            if(character not in punctuations):
                 finalString = finalString + character

        stop_words = set(stopwords.words('english')) 
        word_tokens = word_tokenize(finalString)

        filtered_sentence = []
        for w in word_tokens: 
            if w not in stop_words: 
                #print("Word to be aded")
                #print(w)
                filtered_sentence.append(unicode(w, errors='ignore')) 

        finalStem =[]
        porter = PorterStemmer()
        for words in range(0,len(filtered_sentence)):
            #print(filtered_sentence[words])
            wrd = porter.stem(filtered_sentence[words])
            finalStem.append(wrd)

        for data in (finalStem):
            file1 = open(os.getcwd()+'/'+topic+now.strftime("%Y-%m-%d %H:%M")+'.txt', 'a')
            file1.write(data.encode('utf-8')+' ')
        file1.write('\n\n')
        file1.close()
    print("*********** FILE WRITE COMPLETE ***************") 
    print("*********** TOPIC ENDS HERE ***************")
    print("*********** SLEEPING FOR 10 SECONDS TO STABILIZE ***********")
    time.sleep(10)
        
print("DATA SEARCH ENDS HERE")         
    

      
      

TOPIC NOW: 
molestation
EXTRACTING DATA
100
WRITING DATA TO FILE...
*********** FILE WRITE COMPLETE ***************
*********** TOPIC ENDS HERE ***************
*********** SLEEPING FOR 10 SECONDS TO STABILIZE ***********
TOPIC NOW: 
harassment
EXTRACTING DATA
100
WRITING DATA TO FILE...
*********** FILE WRITE COMPLETE ***************
*********** TOPIC ENDS HERE ***************
*********** SLEEPING FOR 10 SECONDS TO STABILIZE ***********
DATA SEARCH ENDS HERE
